In [30]:
import pandas as pd

## Helper Functions For Plotting

In [31]:
def calculate_speedup_and_efficiency(df: pd.DataFrame, num_threads_column: str = 'num_threads', duration_column: str = 'load_duration_seconds') -> pd.DataFrame:
    """
    Calculates speedup and efficiency for a DataFrame of benchmark results.

    Assumes the DataFrame contains results for different numbers of threads,
    including a run with 1 thread, which serves as the baseline.

    Args:
        df (pd.DataFrame): The input DataFrame containing benchmark results.
                          Must have columns for thread count and duration.
        num_threads_column (str): The name of the column containing the number of threads.
        duration_column (str): The name of the column containing the duration (runtime) in seconds.

    Returns:
        pd.DataFrame: A new DataFrame with 'speedup' and 'efficiency' columns added,
                      or an empty DataFrame if a single-thread baseline is not found.
    """
    if df.empty:
        print("Warning: Input DataFrame is empty.")
        return pd.DataFrame()

    # Get the baseline runtime (1 thread)
    baseline_row = df[df[num_threads_column] == 1]
    if baseline_row.empty:
        print(f"Error: Baseline (1 {num_threads_column}) not found in the DataFrame. Cannot calculate speedup.")
        return pd.DataFrame()

    baseline_runtime = baseline_row[duration_column].iloc[0]

    if baseline_runtime == 0:
        print("Warning: Baseline runtime is zero. Speedup and efficiency calculations will result in division by zero.")
        return pd.DataFrame()

    # Calculate Speedup
    # Speedup = Runtime (1 thread) / Runtime (P threads)
    df['speedup'] = baseline_runtime / df[duration_column]

    # Calculate Efficiency
    # Efficiency = Speedup / Number of threads
    df['efficiency'] = df['speedup'] / df[num_threads_column]

    return df

In [32]:
def extract_efficiency_and_speedup_at_threads(dfs: list[pd.DataFrame], threads = 2):
    extracted_data = []
    for i, df in enumerate(dfs):
        df_at_threads = df[df['num_threads'] == threads]
        efficiency_val = df_at_threads['efficiency'].iloc[0]
        value_size_val = df_at_threads['value_size'].iloc[0]
        speedup_val = df_at_threads['speedup'].iloc[0]

        extracted_data.append({'value_size': value_size_val, f'efficiency_at_{threads}_threads': efficiency_val, f'speedup_at_{threads}_threads': speedup_val })

    result_df = pd.DataFrame(extracted_data)
    result_df = result_df.sort_values(by='value_size').reset_index(drop=True)
    
    return result_df

In [33]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

def plot_workload_performance(df_metrics: pd.DataFrame, threads: int = 2):
    """
    Generates an interactive Plotly graph showing efficiency and speedup
    vs. value_size for a specific number of threads.

    Args:
        df_metrics (pd.DataFrame): DataFrame containing 'value_size',
                                   'efficiency_at_{threads}_threads',
                                   and 'speedup_at_{threads}_threads' columns.
                                   This DataFrame is typically generated by
                                   extract_efficiency_and_speedup_at_threads.
        threads (int): The number of threads for which the data is being plotted.
    """
    if df_metrics.empty:
        print("Error: Input DataFrame is empty. Cannot generate plot.")
        return

    # Check for required columns
    required_efficiency_col = f'efficiency_at_{threads}_threads'
    required_speedup_col = f'speedup_at_{threads}_threads'
    required_cols = ['value_size', required_efficiency_col, required_speedup_col]

    if not all(col in df_metrics.columns for col in required_cols):
        print(f"Error: DataFrame must contain '{required_cols}' columns for plotting.")
        return

    # Define fixed y-axis ranges for consistent comparison
    # Adjust these ranges based on your expected min/max values across all datasets
    # For speedup, it typically starts at 1.0 (baseline) and goes up.
    # For efficiency, it's typically between 0.0 and 1.0 (or slightly above 1.0 for superlinear).
    speedup_y_range = [0.0, 5.0]  # Example: from 0x to 5x speedup
    efficiency_y_range = [0.0, 1.5] # Example: from 0% to 150% efficiency (to accommodate superlinear)

    # Create subplots: one for Speedup, one for Efficiency
    fig = make_subplots(rows=1, cols=2,
                        subplot_titles=(f'Speedup at {threads} Threads vs. Value Size',
                                        f'Efficiency at {threads} Threads vs. Value Size'))

    # Plot Speedup
    fig.add_trace(
        go.Scatter(
            x=df_metrics['value_size'],
            y=df_metrics[required_speedup_col],
            mode='lines+markers',
            name=f'Speedup at {threads} Threads',
            marker=dict(symbol='circle', size=8),
            line=dict(width=2)
        ),
        row=1, col=1
    )
    fig.add_trace(
        go.Scatter(
            x=df_metrics['value_size'],
            y=[1] * len(df_metrics), # Baseline for speedup
            mode='lines',
            name='Ideal (1x Speedup)',
            line=dict(dash='dash', color='gray'),
            showlegend=True
        ),
        row=1, col=1
    )

    # Plot Efficiency
    fig.add_trace(
        go.Scatter(
            x=df_metrics['value_size'],
            y=df_metrics[required_efficiency_col],
            mode='lines+markers',
            name=f'Efficiency at {threads} Threads',
            marker=dict(symbol='diamond', size=8),
            line=dict(width=2)
        ),
        row=1, col=2
    )
    fig.add_trace(
        go.Scatter(
            x=df_metrics['value_size'],
            y=[1] * len(df_metrics), # Ideal efficiency
            mode='lines',
            name='Ideal Efficiency (100%)',
            line=dict(dash='dash', color='gray'),
            showlegend=True
        ),
        row=1, col=2
    )
    fig.add_trace(
        go.Scatter(
            x=df_metrics['value_size'],
            y=[0.5] * len(df_metrics), # 50% efficiency reference
            mode='lines',
            name='50% Efficiency',
            line=dict(dash='dot', color='orange'),
            showlegend=True
        ),
        row=1, col=2
    )

    # Update layout for a beautiful graph
    fig.update_layout(
        title_text=f'Performance Scaling with Workload (at {threads} Threads)',
        height=600, width=1200,
        showlegend=True,
        hovermode="x unified", # Shows hover info for all traces at x-position
        template="plotly_white", # Clean white background
        title_x=0.5, # Center the main title
    )

    # Update x-axes to be consistent and descriptive
    fig.update_xaxes(
        title_text='Value Size (Bytes)',
        type='log', # Use log scale if value_size spans orders of magnitude
        tickmode='array',
        tickvals=df_metrics['value_size'].tolist(), # Set specific tick values
        ticktext=[f'{s}B' for s in df_metrics['value_size'].tolist()], # Add 'B' for bytes
        row=1, col=1
    )
    fig.update_xaxes(
        title_text='Value Size (Bytes)',
        type='log', # Use log scale if value_size spans orders of magnitude
        tickmode='array',
        tickvals=df_metrics['value_size'].tolist(),
        ticktext=[f'{s}B' for s in df_metrics['value_size'].tolist()],
        row=1, col=2
    )

    # Update y-axes for clarity and consistent ranges
    fig.update_yaxes(title_text='Speedup (X times faster)', range=speedup_y_range, row=1, col=1)
    fig.update_yaxes(title_text='Efficiency (Fraction of Ideal)', range=efficiency_y_range, row=1, col=2)

    fig.show()
    # You can also save it as an HTML file for interactive viewing later:
    # fig.write_html(f"performance_scaling_at_{threads}_threads.html")

In [34]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

def plot_single_df_performance(df: pd.DataFrame, workload_description: str = "Performance Metrics"):
    """
    Generates an interactive Plotly graph showing speedup and efficiency
    vs. number of threads for a single DataFrame, with consistent y-axis ranges.

    Args:
        df (pd.DataFrame): The input DataFrame containing 'num_threads', 'speedup',
                           and 'efficiency' columns.
        workload_description (str): A descriptive string for the plot title,
                                    e.g., "Value Size 50 Bytes, No Compression".
    """
    if df.empty or 'num_threads' not in df.columns or 'speedup' not in df.columns or 'efficiency' not in df.columns:
        print("Error: DataFrame must contain 'num_threads', 'speedup', and 'efficiency' columns for plotting.")
        return

    # Define fixed y-axis ranges for consistent comparison across all your plots
    # IMPORTANT: Adjust these ranges based on the MIN and MAX values you expect
    # across ALL your benchmark datasets for both speedup and efficiency.
    speedup_y_range = [0.0, 5.0]  # Example: from 0x to 5x speedup
    efficiency_y_range = [0.0, 1.5] # Example: from 0% to 150% efficiency (to accommodate superlinear)

    # Create subplots: one for Speedup, one for Efficiency
    fig = make_subplots(rows=1, cols=2,
                        subplot_titles=(f'Speedup', f'Efficiency'))

    # Plot Speedup
    fig.add_trace(
        go.Scatter(
            x=df['num_threads'],
            y=df['speedup'],
            mode='lines+markers',
            name='Speedup',
            marker=dict(symbol='circle', size=8),
            line=dict(width=2, color='royalblue')
        ),
        row=1, col=1
    )
    fig.add_trace(
        go.Scatter(
            x=df['num_threads'],
            y=[1] * len(df), # Baseline for speedup
            mode='lines',
            name='Ideal Speedup',
            line=dict(dash='dash', color='gray'),
            showlegend=True
        ),
        row=1, col=1
    )

    # Plot Efficiency
    fig.add_trace(
        go.Scatter(
            x=df['num_threads'],
            y=df['efficiency'],
            mode='lines+markers',
            name='Efficiency',
            marker=dict(symbol='diamond', size=8),
            line=dict(width=2, color='firebrick')
        ),
        row=1, col=2
    )
    fig.add_trace(
        go.Scatter(
            x=df['num_threads'],
            y=[1] * len(df), # Ideal efficiency
            mode='lines',
            name='Ideal Efficiency (100%)',
            line=dict(dash='dash', color='gray'),
            showlegend=True
        ),
        row=1, col=2
    )
    fig.add_trace(
        go.Scatter(
            x=df['num_threads'],
            y=[0.5] * len(df), # 50% efficiency reference
            mode='lines',
            name='50% Efficiency',
            line=dict(dash='dot', color='orange'),
            showlegend=True
        ),
        row=1, col=2
    )

    # Update layout for a beautiful graph
    fig.update_layout(
        title_text=f'{workload_description} vs. Number of Threads',
        height=600, width=1200,
        showlegend=True,
        hovermode="x unified", # Shows hover info for all traces at x-position
        template="plotly_white", # Clean white background
        title_x=0.5, # Center the main title
    )

    # Update x-axes
    fig.update_xaxes(
        title_text='Number of Threads',
        tickmode='array',
        tickvals=df['num_threads'].unique(), # Ensure ticks match thread counts
        row=1, col=1
    )
    fig.update_xaxes(
        title_text='Number of Threads',
        tickmode='array',
        tickvals=df['num_threads'].unique(),
        row=1, col=2
    )

    # Update y-axes for clarity and consistent ranges
    # The key change for consistent axes is setting the 'range' property
    fig.update_yaxes(title_text='Speedup (X times faster)', range=speedup_y_range, row=1, col=1)
    fig.update_yaxes(title_text='Efficiency (Fraction of Ideal)', range=efficiency_y_range, row=1, col=2)

    fig.show()
    # You can also save it as an HTML file for interactive viewing later:
    # fig.write_html(f"performance_metrics_{workload_description.replace(' ', '_').replace('.', '')}.html")

In [35]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px # Import plotly.express for its color sequences

def format_size(size: int) -> str:
    """
    Converts an integer byte value into a human-readable string
    using KB (Kilobytes) and MB (Megabytes) units.
    """
    if size >= 1000000:
        return f'{size / 1000000:.0f}MB'
    elif size >= 1000:
        return f'{size / 1000:.0f}KB'
    else:
        return f'{size}B'
    
    
def plot_combined_workload_performance(dfs: list[pd.DataFrame], workload_type_description: str = "Overall Performance"):
    """
    Generates an interactive Plotly graph showing speedup and efficiency
    vs. value_size for multiple thread counts on the same chart, with distinct colors.

    Args:
        dfs (list[pd.DataFrame]): A list of DataFrames, where each DataFrame
                                   contains 'num_threads', 'value_size', 'speedup',
                                   and 'efficiency' columns for a specific workload.
                                   It's assumed 'speedup' and 'efficiency' have
                                   already been calculated for each df.
        workload_type_description (str): A general description for the plot title,
                                         e.g., "with Compression ON".
    """
    if not dfs:
        print("Error: Input list of DataFrames is empty. Cannot generate plot.")
        return

    # Combine all DataFrames into a single one for easier processing
    combined_df = pd.concat(dfs, ignore_index=True)

    # Ensure required columns exist
    required_cols = ['num_threads', 'value_size', 'speedup', 'efficiency']
    if not all(col in combined_df.columns for col in required_cols):
        print(f"Error: Combined DataFrame must contain '{required_cols}' columns for plotting.")
        return

    # Ensure speedup/efficiency are numeric
    combined_df['speedup'] = pd.to_numeric(combined_df['speedup'], errors='coerce')
    combined_df['efficiency'] = pd.to_numeric(combined_df['efficiency'], errors='coerce')
    combined_df.dropna(subset=['speedup', 'efficiency'], inplace=True)

    # Define fixed y-axis ranges for consistent comparison
    speedup_y_range = [0.0, max(5.0, combined_df['speedup'].max() * 1.1)]
    efficiency_y_range = [0.0, max(1.1, combined_df['efficiency'].max() * 1.1)]

    # Get unique thread counts for different lines
    unique_threads = sorted(combined_df['num_threads'].unique())
    # Get unique value sizes for x-axis ticks
    unique_value_sizes = sorted(combined_df['value_size'].unique())

    # --- Color and Marker Definition ---
    # Using Plotly Express's default qualitative color sequence for distinctness
    colors = px.colors.qualitative.Plotly
    # Define a list of distinct marker symbols
    markers = ['circle', 'square', 'diamond', 'cross', 'x', 'triangle-up', 'triangle-down', 'pentagon', 'hexagram']
    
    # Map thread count to a color and marker
    color_map = {thread: colors[i % len(colors)] for i, thread in enumerate(unique_threads)}
    marker_map = {thread: markers[i % len(markers)] for i, thread in enumerate(unique_threads)}
    # --- End Color and Marker Definition ---


    # Create subplots: one for Speedup, one for Efficiency
    fig = make_subplots(rows=1, cols=2,
                        subplot_titles=(f'Speedup vs. Value Size',
                                        f'Efficiency vs. Value Size'))

    # Plot Speedup for each number of threads
    for threads_count in unique_threads:
        df_subset = combined_df[combined_df['num_threads'] == threads_count]
        fig.add_trace(
            go.Scatter(
                x=df_subset['value_size'],
                y=df_subset['speedup'],
                mode='lines+markers',
                name=f'{threads_count} Threads',
                legendgroup=str(threads_count), # Group traces for consistent legend
                showlegend=True, # Show legend for each thread group
                marker=dict(symbol=marker_map[threads_count], size=8), # Apply distinct marker
                line=dict(width=2, color=color_map[threads_count]) # Apply distinct color
            ),
            row=1, col=1
        )
    # Add ideal speedup line
    fig.add_trace(
        go.Scatter(
            x=unique_value_sizes,
            y=[1] * len(unique_value_sizes),
            mode='lines',
            name='1x Speedup',
            line=dict(dash='dash', color='gray'), # Keep ideal lines distinct and subtle
            showlegend=True,
            legendgroup='ideal_speedup'
        ),
        row=1, col=1
    )


    # Plot Efficiency for each number of threads
    for threads_count in unique_threads:
        df_subset = combined_df[combined_df['num_threads'] == threads_count]
        fig.add_trace(
            go.Scatter(
                x=df_subset['value_size'],
                y=df_subset['efficiency'],
                mode='lines+markers',
                name=f'{threads_count} Threads',
                legendgroup=str(threads_count), # Group traces for consistent legend
                showlegend=False, # Hide legend for efficiency to avoid duplication
                marker=dict(symbol=marker_map[threads_count], size=8), # Apply distinct marker
                line=dict(width=2, color=color_map[threads_count]) # Apply distinct color
            ),
            row=1, col=2
        )
    # Add ideal efficiency lines
    fig.add_trace(
        go.Scatter(
            x=unique_value_sizes,
            y=[1] * len(unique_value_sizes),
            mode='lines',
            name='Ideal Efficiency (100%)',
            line=dict(dash='dash', color='gray'),
            showlegend=True,
            legendgroup='ideal_efficiency_1'
        ),
        row=1, col=2
    )
    fig.add_trace(
        go.Scatter(
            x=unique_value_sizes,
            y=[0.5] * len(unique_value_sizes),
            mode='lines',
            name='50% Efficiency',
            line=dict(dash='dot', color='orange'),
            showlegend=True,
            legendgroup='ideal_efficiency_05'
        ),
        row=1, col=2
    )


    # Update layout for a beautiful graph
    fig.update_layout(
        title_text=workload_type_description,
        height=600, width=1200,
        showlegend=True,
        hovermode="x unified",
        template="plotly_white",
        title_x=0.5,
    )

    # Update x-axes to be consistent and descriptive
    fig.update_xaxes(
        title_text='Value Size (Bytes)',
        type='log',
        tickmode='array',
        tickvals=unique_value_sizes,
        ticktext=[f'{format_size(s)}' for s in unique_value_sizes],
        row=1, col=1
    )
    fig.update_xaxes(
        title_text='Value Size (Bytes)',
        type='log',
        tickmode='array',
        tickvals=unique_value_sizes,
        ticktext=[f'{s}B' for s in unique_value_sizes],
        row=1, col=2
    )

    # Update y-axes for clarity and consistent ranges
    fig.update_yaxes(title_text='Speedup (X times faster)', range=speedup_y_range, row=1, col=1)
    fig.update_yaxes(title_text='Efficiency (Fraction of Ideal)', range=efficiency_y_range, row=1, col=2)

    fig.show()
    # fig.write_html(f"combined_performance_scaling_{workload_type_description.replace(' ', '_').replace('.', '')}.html")

In [36]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px # For distinct colors and markers

def plot_performance_by_workload_on_threads(dfs: list[pd.DataFrame], workload_names: list[str], title: str):
    """
    Generates an interactive Plotly graph showing speedup and efficiency
    vs. number of threads, with each line representing a different workload.

    Args:
        dfs (list[pd.DataFrame]): A list of DataFrames, where each DataFrame
                                   contains 'num_threads', 'value_size', 'speedup',
                                   and 'efficiency' columns for a specific workload.
                                   It's assumed 'speedup' and 'efficiency' have
                                   already been calculated for each df.
        workload_names (list[str]): A list of descriptive names for each workload,
                                     used for legend labels (e.g., ["50 Bytes", "500 Bytes"]).
                                     Length must match dfs.
    """
    if not dfs or not workload_names or len(dfs) != len(workload_names):
        print("Error: Input lists 'dfs' and 'workload_names' must not be empty and must have matching lengths.")
        return

    # Define fixed y-axis ranges for consistent comparison across all your plots
    # Adjust these ranges based on the MIN and MAX values you expect across ALL your datasets.
    speedup_y_range = [0.0, 12]  # Example: from 0x to 5x speedup
    efficiency_y_range = [0.0, 2] # Example: from 0% to 150% efficiency (to accommodate superlinear)

    # Get all unique thread counts across all DFs for x-axis ticks
    all_num_threads = sorted(pd.concat(df['num_threads'] for df in dfs).unique())

    # --- Color and Marker Definition ---
    colors = px.colors.qualitative.Plotly # A good default palette
    markers = ['circle', 'square', 'diamond', 'cross', 'x', 'triangle-up', 'triangle-down', 'pentagon', 'hexagram']
    # --- End Color and Marker Definition ---

    # Create subplots: one for Speedup, one for Efficiency
    fig = make_subplots(rows=1, cols=2,
                        subplot_titles=(f'Speedup vs. Number of Threads',
                                        f'Efficiency vs. Number of Threads'))

    # Plot Speedup for each workload
    for i, df in enumerate(dfs):
        workload_label = workload_names[i]
        color = colors[i % len(colors)]
        marker = markers[i % len(markers)]

        fig.add_trace(
            go.Scatter(
                x=df['num_threads'],
                y=df['speedup'],
                mode='lines+markers',
                name=f'Speedup: {workload_label}',
                marker=dict(symbol=marker, size=8),
                line=dict(width=2, color=color),
                legendgroup=workload_label, # Group traces for consistent legend
                showlegend=True
            ),
            row=1, col=1
        )

    # Add Ideal Speedup line (for reference, only once)
    fig.add_trace(
        go.Scatter(
            x=all_num_threads,
            y=[1] * len(all_num_threads),
            mode='lines',
            name='(1x Speedup)',
            line=dict(dash='dash', color='gray'),
            showlegend=True,
            legendgroup='ideal_speedup'
        ),
        row=1, col=1
    )


    # Plot Efficiency for each workload
    for i, df in enumerate(dfs):
        workload_label = workload_names[i]
        color = colors[i % len(colors)]
        marker = markers[i % len(markers)]

        fig.add_trace(
            go.Scatter(
                x=df['num_threads'],
                y=df['efficiency'],
                mode='lines+markers',
                name=f'Efficiency: {workload_label}',
                marker=dict(symbol=marker, size=8),
                line=dict(width=2, color=color),
                legendgroup=workload_label, # Group traces for consistent legend
                showlegend=False # Hide legend here to avoid duplication, as it's shown for Speedup
            ),
            row=1, col=2
        )

    # Add Ideal Efficiency lines (for reference, only once)
    fig.add_trace(
        go.Scatter(
            x=all_num_threads,
            y=[1] * len(all_num_threads),
            mode='lines',
            name='Ideal Efficiency (100%)',
            line=dict(dash='dash', color='gray'),
            showlegend=True,
            legendgroup='ideal_efficiency_1'
        ),
        row=1, col=2
    )
    fig.add_trace(
        go.Scatter(
            x=all_num_threads,
            y=[0.5] * len(all_num_threads),
            mode='lines',
            name='50% Efficiency',
            line=dict(dash='dot', color='orange'),
            showlegend=True,
            legendgroup='ideal_efficiency_05'
        ),
        row=1, col=2
    )


    # Update layout for a beautiful graph
    fig.update_layout(
        title_text=title,
        height=600, width=1200,
        showlegend=True,
        hovermode="x unified",
        template="plotly_white",
        title_x=0.5,
    )

    # Update x-axes for clarity
    fig.update_xaxes(
        title_text='Number of Threads',
        tickmode='array',
        tickvals=all_num_threads,
        row=1, col=1
    )
    fig.update_xaxes(
        title_text='Number of Threads',
        tickmode='array',
        tickvals=all_num_threads,
        row=1, col=2
    )

    # Update y-axes for clarity and consistent ranges
    fig.update_yaxes(title_text='Speedup (X times faster)', range=speedup_y_range, row=1, col=1)
    fig.update_yaxes(title_text='Efficiency (Fraction of Ideal)', range=efficiency_y_range, row=1, col=2)

    fig.show()
    # fig.write_html(output_filename)
    # You can also save it as an HTML file for interactive viewing later:
    # fig.write_html(f"performance_scaling_across_workloads.html")

In [37]:
import os
os.listdir('./')

['load_summary_3.0keys_5000B_comp-yes_csum-yes.csv',
 'load_summary_3.0keys_20000B_comp-yes_csum-yes.csv',
 'load_summary_0.5keys_100000B_comp-yes_csum-yes.csv',
 'load_summary_50.0keys_500B_comp-yes_csum-yes.csv',
 'load_summary_0.1keys_500000B_comp-yes_csum-yes.csv',
 'load_summary_50.0keys_50B_comp-yes_csum-yes.csv',
 'performance_improvement_vs_value_size.ipynb',
 'load_summary_50.0keys_300B_comp-yes_csum-yes.csv',
 'load_summary_0.1keys_1000000B_comp-yes_csum-yes.csv']

## Loading and Plotting The Speed up and efficiency of the program for differnet thread configs and workloads

In [38]:
test_configs = [
    {"num_keys": 50.0, "value_size": 50},
    {"num_keys": 50.0, "value_size": 300},
    {"num_keys": 50.0, "value_size": 500},
    {"num_keys": 3.0, "value_size": 5000},
    {"num_keys": 3.0, "value_size": 20000},
    {"num_keys": 0.5, "value_size": 100000},
    {"num_keys": 0.1, "value_size": 500000},
    {"num_keys": 0.1, "value_size": 1000000},

    ]


def load_df(test_config: dict, compression_on: bool):
    num_keys = test_config["num_keys"]
    value_size = test_config["value_size"]
    compression_str = "yes" if compression_on else "no"
    df = pd.read_csv(f"load_summary_{num_keys}keys_{value_size}B_comp-yes_csum-yes.csv")
    ret_df = df[df["rdbcompression"] == compression_str].copy()
    return ret_df


dfs_compression_on = [load_df(config, compression_on=True) for config in test_configs]
dfs_compression_on = [calculate_speedup_and_efficiency(df) for df in dfs_compression_on]
dfs_compression_off = [load_df(config, compression_on=False) for config in test_configs]
dfs_compression_off = [calculate_speedup_and_efficiency(df) for df in dfs_compression_off]

print("\n" + "="*50) # Top border
print("                 COMPRESSION ON")
print("="*50 + "\n") # Bottom border

all_workload_names = [f"Value Size {format_size(config["value_size"])}" for config in test_configs]
plot_performance_by_workload_on_threads(dfs_compression_on, all_workload_names, "Speedup / Efficiency Vs. Num Threads for each workload Compression ON")
plot_combined_workload_performance(dfs_compression_on )

print("\n" + "="*50) # Top border
print("                 COMPRESSION OFF")
print("="*50 + "\n") # Bottom border

plot_performance_by_workload_on_threads(dfs_compression_off, all_workload_names, "Speedup / Efficiency Vs. Num Threads for each workload Compression OFF")
plot_combined_workload_performance(dfs_compression_off)
        


                 COMPRESSION ON




                 COMPRESSION OFF



In [54]:
import pandas as pd
import plotly.express as px
import numpy as np

def plot_raw_times_bar_chart(df: pd.DataFrame, workload_name: str):
    """
    Generates a bar chart showing raw save duration vs. number of threads,
    with separate bars for compression ON and OFF for each thread count,
    and displays the plot directly.
    
    Args:
        df (pd.DataFrame): A DataFrame containing 'num_threads', 'load_duration_seconds',
                           and 'rdbcompression' columns for a single workload.
        workload_name (str): The name of the workload for the plot title.
    """
    if df is None or df.empty:
        print("Error: Input DataFrame is empty or invalid. Cannot generate plot.")
        return

    # Ensure required columns are present and numeric where needed
    required_cols = ['num_threads', 'load_duration_seconds', 'rdbcompression']
    if not all(col in df.columns for col in required_cols):
        print(f"Error: DataFrame must contain '{required_cols}' columns for plotting.")
        return
    
    df['load_duration_seconds'] = pd.to_numeric(df['load_duration_seconds'], errors='coerce')
    df['num_threads'] = df['num_threads'].astype(str)
    df['load_duration_minutes'] = df['load_duration_seconds'] / 60


    fig = px.bar(
        df,
        x='num_threads',
        y='load_duration_seconds',
        color='rdbcompression',
        barmode='group',
        title=f'Raw Save Times for Workload: {workload_name}',
        labels={'num_threads': 'Number of Threads', 'load_duration_seconds': 'Load Duration (Minutes)', 'rdbcompression': 'Compression'},
        template="plotly_white",
    )

    fig.update_layout(
        title_x=0.5,
        xaxis_title="Number of Threads",
        yaxis_title="Load Duration (seconds)",
        legend_title="RDB Compression",
    )

    fig.show()


## Plotting the Raw Save Time for each workload for each thread configuration

In [56]:
combined_dfs = []
for i in range(len(dfs_compression_on)):
    combined_df = pd.concat([dfs_compression_on[i], dfs_compression_off[i]], ignore_index=True)
    combined_dfs.append(combined_df)

for df in combined_dfs:
    if not df.empty:
        num_keys = f"{round(df['keys'].iloc[0], 1)}"
        value_size = format_size(df['value_size'].iloc[0])
        workload_name = f"Num Keys: {num_keys} Million, Value Size: {format_size(df['value_size'].iloc[0])}"
        
        plot_raw_times_bar_chart(df, workload_name)
        

In [41]:
def print_raw_data(df):
    print(f"Num Keys: {df['keys'].iloc[0]}, Value Size: {df['value_size'].iloc[0]} Bytes ")
    return df

In [42]:
for df_comp_on, df_comp_off in zip(dfs_compression_on, dfs_compression_off):
    display(print_raw_data(df_comp_on))
    display(print_raw_data(df_comp_off))


Num Keys: 50.0, Value Size: 50 Bytes 


,num_threads,keys,value_size,rdbcompression,rdbchecksum,actual_throughput_mb_s,rdb_file_size_bytes,load_duration_seconds,avg_cpu_percent,avg_memory_rss_bytes,total_disk_read_bytes,total_disk_write_bytes,cpu_user_time_seconds,cpu_system_time_seconds,memory_rss_bytes_final,status,flamegraph_generated,speedup,efficiency
0,1,50.0,50,yes,yes,84.611139,3450019200,40.775,96.364634,2.947557e+09,3429875712,4096,33.39,6.64,5529157632,ok,False,1.000000,1.000000
1,2,50.0,50,yes,yes,160.930087,3450019200,21.438,139.490909,2.981698e+09,3450019840,0,24.78,6.09,5557129216,ok,False,1.901996,0.950998
2,3,50.0,50,yes,yes,168.986050,3450019200,20.416,161.804878,3.004638e+09,3450019840,4096,28.55,5.79,5600047104,ok,False,1.997208,0.665736
3,4,50.0,50,yes,yes,153.375087,3450019200,22.494,164.852174,3.115031e+09,3450019840,0,31.48,6.68,5622636544,ok,False,1.812706,0.453176
4,6,50.0,50,yes,yes,145.705685,3450019200,23.678,165.731250,3.147007e+09,3450019840,4096,34.04,6.67,5675114496,ok,False,1.722063,0.287010
5,8,50.0,50,yes,yes,161.775260,3450019200,21.326,167.383721,3.187868e+09,3450019840,0,30.85,6.74,5672484864,ok,False,1.911985,0.238998
6,10,50.0,50,yes,yes,151.582566,3450019200,22.760,170.880435,3.275212e+09,3450019840,0,33.93,7.20,5673943040,ok,False,1.791520,0.179152
7,15,50.0,50,yes,yes,146.044922,3450019200,23.623,183.631250,3.450410e+09,3450019840,0,37.99,7.82,5666488320,ok,False,1.726072,0.115071


Num Keys: 50.0, Value Size: 50 Bytes 


,num_threads,keys,value_size,rdbcompression,rdbchecksum,actual_throughput_mb_s,rdb_file_size_bytes,load_duration_seconds,avg_cpu_percent,avg_memory_rss_bytes,total_disk_read_bytes,total_disk_write_bytes,cpu_user_time_seconds,cpu_system_time_seconds,memory_rss_bytes_final,status,flamegraph_generated,speedup,efficiency
8,1,50.0,50,no,yes,91.130519,3450019200,37.858,96.532895,2.943015e+09,3422011392,4096,31.62,5.58,5528989696,ok,False,1.000000,1.000000
9,2,50.0,50,no,yes,146.298838,3450019200,23.582,135.652083,3.046149e+09,3450019840,4096,26.44,6.52,5563936768,ok,False,1.605377,0.802688
10,3,50.0,50,no,yes,173.778230,3450019200,19.853,166.550000,3.007038e+09,3449536512,0,27.95,6.36,5601349632,ok,False,1.906916,0.635639
11,4,50.0,50,no,yes,176.588995,3450019200,19.537,172.450000,3.080893e+09,3450019840,4096,28.41,6.43,5622128640,ok,False,1.937759,0.484440
12,6,50.0,50,no,yes,155.609544,3450019200,22.171,170.908889,3.099943e+09,3450019840,0,32.54,6.87,5679988736,ok,False,1.707546,0.284591
13,8,50.0,50,no,yes,155.287357,3450019200,22.217,165.897778,3.142558e+09,3450019840,4096,31.05,7.16,5695631360,ok,False,1.704010,0.213001
14,10,50.0,50,no,yes,156.513142,3450019200,22.043,171.755556,3.280169e+09,3450019840,0,32.47,6.73,5679992832,ok,False,1.717461,0.171746
15,15,50.0,50,no,yes,150.985523,3450019200,22.850,181.797826,3.388148e+09,3450019840,0,36.81,7.96,5668548608,ok,False,1.656805,0.110454


Num Keys: 50.0, Value Size: 300 Bytes 


...

,num_threads,keys,value_size,rdbcompression,rdbchecksum,actual_throughput_mb_s,rdb_file_size_bytes,load_duration_seconds,avg_cpu_percent,avg_memory_rss_bytes,total_disk_read_bytes,total_disk_write_bytes,cpu_user_time_seconds,cpu_system_time_seconds,memory_rss_bytes_final,status,flamegraph_generated,speedup,efficiency
8,1,0.1,500000,no,yes,392.641919,50002555756,127.349,55.320784,2.523236e+10,49913511936,4096,14.84,56.13,50516971520,ok,False,1.000000,1.000000
9,2,0.1,500000,no,yes,400.308668,50002555756,124.910,144.811600,2.512324e+10,49883365376,8192,124.94,56.82,50528784384,ok,False,1.019526,0.509763
10,3,0.1,500000,no,yes,402.931221,50002555756,124.097,245.135484,2.527586e+10,49808916480,4096,249.18,56.26,50528813056,ok,False,1.026205,0.342068
11,4,0.1,500000,no,yes,415.079531,50002555756,120.465,344.341494,2.504676e+10,49856102400,8192,360.45,56.24,50527399936,ok,False,1.057145,0.264286
12,6,0.1,500000,no,yes,413.811971,50002555756,120.834,543.081818,2.524502e+10,49921900544,4096,602.08,57.15,50528133120,ok,False,1.053917,0.175653
13,8,0.1,500000,no,yes,411.824998,50002555756,121.417,742.742387,2.513485e+10,49879957504,8192,848.17,57.60,50528808960,ok,False,1.048856,0.131107
14,10,0.1,500000,no,yes,441.636761,50002555756,113.221,934.610132,2.509830e+10,49954930688,4096,1015.42,47.83,50528796672,ok,False,1.124783,0.112478
15,15,0.1,500000,no,yes,385.361302,50002555756,129.755,1444.036154,2.527651e+10,49956503552,4096,1816.95,64.38,50529386496,ok,False,0.981457,0.065430


Num Keys: 0.1, Value Size: 1000000 Bytes 


,num_threads,keys,value_size,rdbcompression,rdbchecksum,actual_throughput_mb_s,rdb_file_size_bytes,load_duration_seconds,avg_cpu_percent,avg_memory_rss_bytes,total_disk_read_bytes,total_disk_write_bytes,cpu_user_time_seconds,cpu_system_time_seconds,memory_rss_bytes_final,status,flamegraph_generated,speedup,efficiency
0,1,0.1,1000000,yes,yes,159.527929,75808309849,475.204,82.754842,5.233800e+10,75760123904,4096,272.26,121.75,100592865280,ok,False,1.000000,1.000000
1,2,0.1,1000000,yes,yes,317.124564,75808309849,239.049,154.292469,4.948291e+10,75744133120,4096,288.04,81.78,101203169280,ok,False,1.987894,0.993947
2,3,0.1,1000000,yes,yes,434.972486,75808309849,174.283,237.188252,4.921185e+10,75745443840,8192,335.27,79.70,100614627328,ok,False,2.726623,0.908874
3,4,0.1,1000000,yes,yes,424.206853,75808309849,178.706,337.466760,4.936800e+10,75782144000,4096,525.98,79.23,100614623232,ok,False,2.659138,0.664785
4,6,0.1,1000000,yes,yes,420.209471,75808309849,180.406,536.315512,4.959248e+10,75702452224,8192,890.82,79.65,100606533632,ok,False,2.634081,0.439013
5,8,0.1,1000000,yes,yes,421.923772,75808309849,179.673,735.683333,4.931578e+10,75799707648,4096,1246.41,79.58,100615184384,ok,False,2.644827,0.330603
6,10,0.1,1000000,yes,yes,415.654474,75808309849,182.383,935.369315,4.893413e+10,75714772992,4096,1631.47,79.45,100618473472,ok,False,2.605528,0.260553
7,15,0.1,1000000,yes,yes,413.259430,75808309849,183.440,1434.868392,4.919587e+10,75688558592,4096,2559.88,79.60,100613017600,ok,False,2.590515,0.172701


Num Keys: 0.1, Value Size: 1000000 Bytes 


,num_threads,keys,value_size,rdbcompression,rdbchecksum,actual_throughput_mb_s,rdb_file_size_bytes,load_duration_seconds,avg_cpu_percent,avg_memory_rss_bytes,total_disk_read_bytes,total_disk_write_bytes,cpu_user_time_seconds,cpu_system_time_seconds,memory_rss_bytes_final,status,flamegraph_generated,speedup,efficiency
8,1,0.1,1000000,no,yes,426.977214,100002760204,234.211,48.046695,4.987870e+10,99983278080,4096,28.72,84.42,100591616000,ok,False,1.000000,1.000000
9,2,0.1,1000000,no,yes,419.899144,100002760204,238.159,143.287841,5.046935e+10,100002762752,12288,230.26,112.21,100607905792,ok,False,0.983423,0.491711
10,3,0.1,1000000,no,yes,413.675572,100002760204,241.742,242.013017,5.003603e+10,99977773056,4096,478.93,107.80,100606910464,ok,False,0.968847,0.322949
11,4,0.1,1000000,no,yes,435.398488,100002760204,229.681,337.370870,5.006619e+10,100001366016,8192,681.80,95.32,100608622592,ok,False,1.019723,0.254931
12,6,0.1,1000000,no,yes,434.788938,100002760204,230.003,536.890870,5.016398e+10,99859808256,4096,1144.17,93.56,100607930368,ok,False,1.018295,0.169716
13,8,0.1,1000000,no,yes,443.933857,100002760204,225.265,737.224390,4.982831e+10,99961257984,8192,1571.02,93.81,100608086016,ok,False,1.039713,0.129964
14,10,0.1,1000000,no,yes,439.762008,100002760204,227.402,937.241099,5.052411e+10,99906994176,4096,2041.24,95.07,100604375040,ok,False,1.029943,0.102994
15,15,0.1,1000000,no,yes,402.548708,100002760204,248.424,1442.509054,5.200099e+10,99920625664,4096,3478.14,112.92,100603314176,ok,False,0.942787,0.062852


In [43]:
# 2 Billion 50 Byte Keys, Compression OFF
df = dfs_compression_off[0]
print(f"Num Keys: {df['keys'].iloc[0]}, Value Size: {df['value_size'].iloc[0]} Bytes ")
dfs_compression_off[0]

Num Keys: 50.0, Value Size: 50 Bytes 


,num_threads,keys,value_size,rdbcompression,rdbchecksum,actual_throughput_mb_s,rdb_file_size_bytes,load_duration_seconds,avg_cpu_percent,avg_memory_rss_bytes,total_disk_read_bytes,total_disk_write_bytes,cpu_user_time_seconds,cpu_system_time_seconds,memory_rss_bytes_final,status,flamegraph_generated,speedup,efficiency
8,1,50.0,50,no,yes,91.130519,3450019200,37.858,96.532895,2.943015e+09,3422011392,4096,31.62,5.58,5528989696,ok,False,1.000000,1.000000
9,2,50.0,50,no,yes,146.298838,3450019200,23.582,135.652083,3.046149e+09,3450019840,4096,26.44,6.52,5563936768,ok,False,1.605377,0.802688
10,3,50.0,50,no,yes,173.778230,3450019200,19.853,166.550000,3.007038e+09,3449536512,0,27.95,6.36,5601349632,ok,False,1.906916,0.635639
11,4,50.0,50,no,yes,176.588995,3450019200,19.537,172.450000,3.080893e+09,3450019840,4096,28.41,6.43,5622128640,ok,False,1.937759,0.484440
12,6,50.0,50,no,yes,155.609544,3450019200,22.171,170.908889,3.099943e+09,3450019840,0,32.54,6.87,5679988736,ok,False,1.707546,0.284591
13,8,50.0,50,no,yes,155.287357,3450019200,22.217,165.897778,3.142558e+09,3450019840,4096,31.05,7.16,5695631360,ok,False,1.704010,0.213001
14,10,50.0,50,no,yes,156.513142,3450019200,22.043,171.755556,3.280169e+09,3450019840,0,32.47,6.73,5679992832,ok,False,1.717461,0.171746
15,15,50.0,50,no,yes,150.985523,3450019200,22.850,181.797826,3.388148e+09,3450019840,0,36.81,7.96,5668548608,ok,False,1.656805,0.110454


In [44]:
# 1 Billion 300 Byte Keys, Compression ON
dfs_compression_on[1]

,num_threads,keys,value_size,rdbcompression,rdbchecksum,actual_throughput_mb_s,rdb_file_size_bytes,load_duration_seconds,avg_cpu_percent,avg_memory_rss_bytes,total_disk_read_bytes,total_disk_write_bytes,cpu_user_time_seconds,cpu_system_time_seconds,memory_rss_bytes_final,status,flamegraph_generated,speedup,efficiency
0,1,50.0,300,yes,yes,185.314859,16000084883,86.340,83.102890,9.672907e+09,15950004224,4096,49.35,23.08,18842271744,ok,False,1.000000,1.000000
1,2,50.0,300,yes,yes,392.659391,16000084883,40.748,140.729268,9.526358e+09,15918940160,0,39.54,18.79,18851794944,ok,False,2.118877,1.059439
2,3,50.0,300,yes,yes,408.822467,16000084883,39.137,232.259494,9.476258e+09,15956951040,4096,74.37,18.10,18852069376,ok,False,2.206097,0.735366
3,4,50.0,300,yes,yes,410.690338,16000084883,38.959,324.735443,9.620808e+09,16000086016,4096,111.40,17.46,18852261888,ok,False,2.216176,0.554044
4,6,50.0,300,yes,yes,412.033500,16000084883,38.832,501.243590,9.662518e+09,15888531456,4096,178.35,19.12,18860904448,ok,False,2.223424,0.370571
5,8,50.0,300,yes,yes,414.789363,16000084883,38.574,695.825641,9.635624e+09,15990243328,0,254.33,18.05,18857889792,ok,False,2.238295,0.279787
6,10,50.0,300,yes,yes,416.647177,16000084883,38.402,869.666234,9.525016e+09,15848423424,0,320.49,18.20,18857754624,ok,False,2.248320,0.224832
7,15,50.0,300,yes,yes,394.391897,16000084883,40.569,1343.692683,9.501181e+09,15996534784,0,533.26,19.07,18863140864,ok,False,2.128226,0.141882


In [45]:
# 1 Billion 300 Byte Keys, Compression OFF
dfs_compression_off[1]

,num_threads,keys,value_size,rdbcompression,rdbchecksum,actual_throughput_mb_s,rdb_file_size_bytes,load_duration_seconds,avg_cpu_percent,avg_memory_rss_bytes,total_disk_read_bytes,total_disk_write_bytes,cpu_user_time_seconds,cpu_system_time_seconds,memory_rss_bytes_final,status,flamegraph_generated,speedup,efficiency
8,1,50.0,300,no,yes,195.719731,16000087971,81.750,80.669512,9.649804e+09,15964160000,4096,48.49,18.20,18842378240,ok,False,1.000000,1.000000
9,2,50.0,300,no,yes,424.337983,16000087971,37.706,133.397368,9.436141e+09,15932047360,4096,35.76,15.53,18856009728,ok,False,2.168090,1.084045
10,3,50.0,300,no,yes,410.037876,16000087971,39.021,233.700000,9.629256e+09,16000090112,0,75.78,17.08,18855931904,ok,False,2.095026,0.698342
11,4,50.0,300,no,yes,407.520961,16000087971,39.262,324.441772,9.596532e+09,15910289408,4096,111.89,17.42,18856210432,ok,False,2.082166,0.520541
12,6,50.0,300,no,yes,395.493573,16000087971,40.456,512.597561,9.599430e+09,16000090112,4096,191.27,19.57,18852577280,ok,False,2.020714,0.336786
13,8,50.0,300,no,yes,401.145464,16000087971,39.886,702.768750,9.519839e+09,15867035648,4096,264.28,19.83,18857541632,ok,False,2.049591,0.256199
14,10,50.0,300,no,yes,392.139796,16000087971,40.802,888.119512,9.530100e+09,15897968640,0,347.50,19.70,18853376000,ok,False,2.003578,0.200358
15,15,50.0,300,no,yes,403.055344,16000087971,39.697,1332.205000,9.570169e+09,15942008832,0,517.01,19.00,18858639360,ok,False,2.059350,0.137290


In [46]:
dfs_compression_on[2]

,num_threads,keys,value_size,rdbcompression,rdbchecksum,actual_throughput_mb_s,rdb_file_size_bytes,load_duration_seconds,avg_cpu_percent,avg_memory_rss_bytes,total_disk_read_bytes,total_disk_write_bytes,cpu_user_time_seconds,cpu_system_time_seconds,memory_rss_bytes_final,status,flamegraph_generated,speedup,efficiency
0,1,50.0,500,yes,yes,210.380268,25999635015,123.584,74.716935,1.489013e+10,25999638528,4096,62.99,30.24,29160861696,ok,False,1.000000,1.000000
1,2,50.0,500,yes,yes,412.247654,25999635015,63.068,138.357480,1.473561e+10,25992609792,0,62.60,25.82,29174755328,ok,False,1.959536,0.979768
2,3,50.0,500,yes,yes,405.446075,25999635015,64.126,236.617054,1.500895e+10,25969016832,4096,123.66,29.70,29175218176,ok,False,1.927206,0.642402
3,4,50.0,500,yes,yes,400.062088,25999635015,64.989,335.527692,1.453178e+10,25817497600,0,189.74,30.10,29175046144,ok,False,1.901614,0.475404
4,6,50.0,500,yes,yes,419.945003,25999635015,61.912,525.296774,1.473813e+10,25841352704,4096,299.10,28.94,29176000512,ok,False,1.996124,0.332687
5,8,50.0,500,yes,yes,409.630146,25999635015,63.471,718.763780,1.456204e+10,25825886208,0,430.51,29.46,29172199424,ok,False,1.947094,0.243387
6,10,50.0,500,yes,yes,393.302197,25999635015,66.106,915.045113,1.461680e+10,25983434752,0,580.47,29.50,29176471552,ok,False,1.869482,0.186948
7,15,50.0,500,yes,yes,383.384968,25999635015,67.816,1404.722794,1.430240e+10,25891684352,0,927.06,32.90,29173215232,ok,False,1.822343,0.121490


In [47]:
dfs_compression_off[2]

,num_threads,keys,value_size,rdbcompression,rdbchecksum,actual_throughput_mb_s,rdb_file_size_bytes,load_duration_seconds,avg_cpu_percent,avg_memory_rss_bytes,total_disk_read_bytes,total_disk_write_bytes,cpu_user_time_seconds,cpu_system_time_seconds,memory_rss_bytes_final,status,flamegraph_generated,speedup,efficiency
8,1,50.0,500,no,yes,205.096969,26000142803,126.770,77.477165,1.489731e+10,25965477888,4096,65.10,33.87,29161259008,ok,False,1.000000,1.000000
9,2,50.0,500,no,yes,394.479484,26000142803,65.910,142.900758,1.477350e+10,25861799936,4096,65.87,29.16,29174824960,ok,False,1.923380,0.961690
10,3,50.0,500,no,yes,396.863919,26000142803,65.514,239.512121,1.471572e+10,26000146432,4096,129.21,29.50,29170724864,ok,False,1.935006,0.645002
11,4,50.0,500,no,yes,410.894050,26000142803,63.277,333.531496,1.464872e+10,25911083008,4096,184.10,28.90,29183492096,ok,False,2.003414,0.500853
12,6,50.0,500,no,yes,406.996271,26000142803,63.883,529.692969,1.466765e+10,25863372800,0,309.38,31.88,29176246272,ok,False,1.984409,0.330735
13,8,50.0,500,no,yes,403.365646,26000142803,64.458,723.892248,1.466371e+10,25844760576,4096,438.45,31.99,29180964864,ok,False,1.966707,0.245838
14,10,50.0,500,no,yes,402.335744,26000142803,64.623,918.137692,1.487888e+10,25983434752,0,563.97,34.38,29180870656,ok,False,1.961685,0.196169
15,15,50.0,500,no,yes,362.406685,26000142803,71.743,1410.193750,1.441223e+10,25940705280,0,982.10,37.05,29177524224,ok,False,1.767002,0.117800


In [48]:
dfs_compression_on[3]

,num_threads,keys,value_size,rdbcompression,rdbchecksum,actual_throughput_mb_s,rdb_file_size_bytes,load_duration_seconds,avg_cpu_percent,avg_memory_rss_bytes,total_disk_read_bytes,total_disk_write_bytes,cpu_user_time_seconds,cpu_system_time_seconds,memory_rss_bytes_final,status,flamegraph_generated,speedup,efficiency
0,1,3.0,5000,yes,yes,180.821926,13536510168,74.861,89.861333,7.792331e+09,13476937728,4096,53.82,14.12,15663984640,ok,False,1.000000,1.000000
1,2,3.0,5000,yes,yes,316.192338,13536510168,42.811,151.994186,7.832796e+09,13487292416,0,52.37,13.75,15694888960,ok,False,1.748639,0.874320
2,3,3.0,5000,yes,yes,401.915385,13536510168,33.680,238.038235,7.876865e+09,13478641664,4096,66.18,15.56,15680716800,ok,False,2.222714,0.740905
3,4,3.0,5000,yes,yes,393.057585,13536510168,34.439,336.957971,7.736903e+09,13380337664,0,102.87,15.00,15679004672,ok,False,2.173727,0.543432
4,6,3.0,5000,yes,yes,393.709213,13536510168,34.382,534.669565,7.745880e+09,13395542016,4096,171.93,14.78,15678992384,ok,False,2.177331,0.362889
5,8,3.0,5000,yes,yes,397.046613,13536510168,34.093,731.295652,7.804775e+09,13517438976,0,237.91,15.52,15675445248,ok,False,2.195788,0.274473
6,10,3.0,5000,yes,yes,387.056019,13536510168,34.973,928.133803,7.845902e+09,13536514048,8192,313.99,16.25,15681650688,ok,False,2.140537,0.214054
7,15,3.0,5000,yes,yes,365.456538,13536510168,37.040,1428.066667,7.857700e+09,13536514048,0,517.91,18.72,15681249280,ok,False,2.021085,0.134739


In [49]:
dfs_compression_off[3]

,num_threads,keys,value_size,rdbcompression,rdbchecksum,actual_throughput_mb_s,rdb_file_size_bytes,load_duration_seconds,avg_cpu_percent,avg_memory_rss_bytes,total_disk_read_bytes,total_disk_write_bytes,cpu_user_time_seconds,cpu_system_time_seconds,memory_rss_bytes_final,status,flamegraph_generated,speedup,efficiency
8,1,3.0,5000,no,yes,266.786231,15060082727,56.450,47.982301,7.680742e+09,14946648064,4096,12.93,14.47,15664852992,ok,False,1.000000,1.000000
9,2,3.0,5000,no,yes,421.013747,15060082727,35.771,136.593056,7.855830e+09,14972338176,4096,35.62,14.17,15675949056,ok,False,1.578094,0.789047
10,3,3.0,5000,no,yes,415.852070,15060082727,36.215,236.269863,7.793735e+09,14978629632,0,73.44,13.64,15670988800,ok,False,1.558746,0.519582
11,4,3.0,5000,no,yes,415.656953,15060082727,36.232,334.208219,7.894543e+09,14974435328,4096,108.99,14.09,15675985920,ok,False,1.558015,0.389504
12,6,3.0,5000,no,yes,399.736768,15060082727,37.675,533.951316,7.817909e+09,15007727616,0,188.69,15.54,15676493824,ok,False,1.498341,0.249724
13,8,3.0,5000,no,yes,402.826800,15060082727,37.386,732.934667,7.768159e+09,14920433664,4096,262.23,15.65,15676592128,ok,False,1.509924,0.188740
14,10,3.0,5000,no,yes,394.439190,15060082727,38.181,930.487013,7.876221e+09,15008514048,0,343.58,16.72,15675068416,ok,False,1.478484,0.147848
15,15,3.0,5000,no,yes,378.736614,15060082727,39.764,1426.836250,7.837904e+09,14984658944,0,556.71,18.08,15677038592,ok,False,1.419626,0.094642


In [50]:
dfs_compression_on[4]

,num_threads,keys,value_size,rdbcompression,rdbchecksum,actual_throughput_mb_s,rdb_file_size_bytes,load_duration_seconds,avg_cpu_percent,avg_memory_rss_bytes,total_disk_read_bytes,total_disk_write_bytes,cpu_user_time_seconds,cpu_system_time_seconds,memory_rss_bytes_final,status,flamegraph_generated,speedup,efficiency
0,1,3.0,20000,yes,yes,173.741705,47829875121,275.293,91.914701,3.161937e+10,47824093184,4096,185.95,68.05,62123405312,ok,False,1.000000,1.000000
1,2,3.0,20000,yes,yes,304.627543,47829875121,157.011,154.823567,3.090203e+10,47759474688,4096,190.66,53.43,62162202624,ok,False,1.753336,0.876668
2,3,3.0,20000,yes,yes,389.136016,47829875121,122.913,241.548780,3.099115e+10,47713599488,8192,244.90,53.43,62137487360,ok,False,2.239739,0.746580
3,4,3.0,20000,yes,yes,381.820378,47829875121,125.268,341.638247,3.073290e+10,47778086912,4096,373.05,57.00,62136610816,ok,False,2.197632,0.549408
4,6,3.0,20000,yes,yes,389.643228,47829875121,122.753,541.350000,3.110625e+10,47781494784,8192,608.30,59.32,62140801024,ok,False,2.242658,0.373776
5,8,3.0,20000,yes,yes,390.958600,47829875121,122.340,740.982857,3.094631e+10,47747416064,4096,850.80,59.73,62138003456,ok,False,2.250229,0.281279
6,10,3.0,20000,yes,yes,354.203541,47829875121,135.035,937.417778,3.077157e+10,47675588608,4096,1211.16,59.52,62137413632,ok,False,2.038679,0.203868
7,15,3.0,20000,yes,yes,328.019773,47829875121,145.814,1439.363699,3.088417e+10,47781232640,4096,2038.32,67.99,62138613760,ok,False,1.887974,0.125865


In [51]:
dfs_compression_off[4]

,num_threads,keys,value_size,rdbcompression,rdbchecksum,actual_throughput_mb_s,rdb_file_size_bytes,load_duration_seconds,avg_cpu_percent,avg_memory_rss_bytes,total_disk_read_bytes,total_disk_write_bytes,cpu_user_time_seconds,cpu_system_time_seconds,memory_rss_bytes_final,status,flamegraph_generated,speedup,efficiency
8,1,3.0,20000,no,yes,261.415361,60069328804,229.785,47.515217,3.139297e+10,60043804672,4096,32.85,76.90,62123593728,ok,False,1.000000,1.000000
9,2,3.0,20000,no,yes,430.505754,60069328804,139.532,135.562366,3.084816e+10,59891499008,8192,135.49,54.41,62133022720,ok,False,1.646827,0.823413
10,3,3.0,20000,no,yes,395.096778,60069328804,152.037,241.575410,3.107857e+10,60069330944,8192,303.92,65.29,62136848384,ok,False,1.511376,0.503792
11,4,3.0,20000,no,yes,423.931013,60069328804,141.696,335.427465,3.076979e+10,60037775360,8192,421.41,56.02,62131752960,ok,False,1.621676,0.405419
12,6,3.0,20000,no,yes,396.719802,60069328804,151.415,540.890429,3.117541e+10,59962277888,4096,754.29,67.64,62133407744,ok,False,1.517584,0.252931
13,8,3.0,20000,no,yes,390.707527,60069328804,153.745,741.045779,3.119972e+10,60032532480,8192,1075.01,68.42,62132867072,ok,False,1.494585,0.186823
14,10,3.0,20000,no,yes,394.728108,60069328804,152.179,940.002295,3.097790e+10,60055339008,4096,1366.67,69.00,62132875264,ok,False,1.509965,0.150997
15,15,3.0,20000,no,yes,402.930815,60069328804,149.081,1434.265101,3.066534e+10,59903033344,4096,2082.89,62.57,62138441728,ok,False,1.541343,0.102756


In [52]:
dfs_compression_on[5]


,num_threads,keys,value_size,rdbcompression,rdbchecksum,actual_throughput_mb_s,rdb_file_size_bytes,load_duration_seconds,avg_cpu_percent,avg_memory_rss_bytes,total_disk_read_bytes,total_disk_write_bytes,cpu_user_time_seconds,cpu_system_time_seconds,memory_rss_bytes_final,status,flamegraph_generated,speedup,efficiency
0,1,0.5,100000,yes,yes,179.983386,38264467770,212.600,83.143059,2.570160e+10,38181912576,4096,139.47,37.85,51421270016,ok,False,1.000000,1.000000
1,2,0.5,100000,yes,yes,305.973770,38264467770,125.058,153.034400,2.621421e+10,38165004288,4096,146.45,45.78,52196417536,ok,False,1.700011,0.850006
2,3,0.5,100000,yes,yes,378.185866,38264467770,101.179,241.735468,2.581142e+10,38238142464,8192,198.59,47.61,51441553408,ok,False,2.101227,0.700409
3,4,0.5,100000,yes,yes,378.530056,38264467770,101.087,342.032020,2.566694e+10,38264471552,8192,299.20,48.77,51441750016,ok,False,2.103139,0.525785
4,6,0.5,100000,yes,yes,380.007426,38264467770,100.694,539.735149,2.579121e+10,38232113152,8192,500.23,46.39,51441131520,ok,False,2.111347,0.351891
5,8,0.5,100000,yes,yes,378.541290,38264467770,101.084,737.951232,2.561238e+10,38264471552,4096,703.70,46.48,51437363200,ok,False,2.103201,0.262900
6,10,0.5,100000,yes,yes,381.218919,38264467770,100.374,937.934826,2.561807e+10,38158188544,4096,899.84,46.58,51442151424,ok,False,2.118078,0.211808
7,15,0.5,100000,yes,yes,357.358024,38264467770,107.076,1438.265581,2.557495e+10,38264471552,8192,1496.07,52.05,51439431680,ok,False,1.985506,0.132367


In [53]:
dfs_compression_off[5]


,num_threads,keys,value_size,rdbcompression,rdbchecksum,actual_throughput_mb_s,rdb_file_size_bytes,load_duration_seconds,avg_cpu_percent,avg_memory_rss_bytes,total_disk_read_bytes,total_disk_write_bytes,cpu_user_time_seconds,cpu_system_time_seconds,memory_rss_bytes_final,status,flamegraph_generated,speedup,efficiency
8,1,0.5,100000,no,yes,269.187321,50011774018,185.788,41.741667,2.582544e+10,49975377920,4096,17.87,60.18,51420663808,ok,False,1.000000,1.000000
9,2,0.5,100000,no,yes,385.132562,50011774018,129.856,143.022692,2.569943e+10,49914036224,8192,130.20,56.48,51431903232,ok,False,1.430723,0.715362
10,3,0.5,100000,no,yes,383.143906,50011774018,130.530,242.443295,2.578888e+10,49852694528,4096,261.86,55.90,51432517632,ok,False,1.423336,0.474445
11,4,0.5,100000,no,yes,390.436358,50011774018,128.092,342.391440,2.594867e+10,50011774976,12288,383.91,56.91,51436363776,ok,False,1.450426,0.362607
12,6,0.5,100000,no,yes,384.978400,50011774018,129.908,540.665385,2.557968e+10,49891491840,4096,648.25,57.07,51432902656,ok,False,1.430151,0.238358
13,8,0.5,100000,no,yes,390.998014,50011774018,127.908,739.293750,2.566734e+10,49892540416,8192,893.55,56.02,51433160704,ok,False,1.452513,0.181564
14,10,0.5,100000,no,yes,395.028349,50011774018,126.603,937.458268,2.543803e+10,50011774976,8192,1136.66,55.58,51437387776,ok,False,1.467485,0.146748
15,15,0.5,100000,no,yes,411.768657,50011774018,121.456,1430.064609,2.549000e+10,49875763200,4096,1696.80,47.45,51433906176,ok,False,1.529673,0.101978
